In [2]:
pip install backtrader

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 419 kB 1.8 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from openbb import obb

# Set your API key
obb.user.credentials.fmp_api_key = "JI198J9ADpdcD2yJ4kd8Zl0RAP1goenk"

# Parameters
symbol = "AAPL"
start_date = "2024-01-01"
short_window = 20
long_window = 50
risk_free_rate = 0.05

# Fetch data
data = obb.equity.price.historical(
    symbol=symbol,
    start_date=start_date,
    provider="fmp"
).to_df()

# Preprocess
data.index = pd.to_datetime(data.index)
data = data.sort_index()
price_col = 'close'

# SMAC logic
data['Short MA'] = data[price_col].rolling(window=short_window).mean()
data['Long MA'] = data[price_col].rolling(window=long_window).mean()
data['Signal'] = 0
data['Signal'][long_window:] = np.where(
    data['Short MA'][long_window:] > data['Long MA'][long_window:], 1, 0
)
data['Position'] = data['Signal'].diff()

# Returns
data['Daily Return'] = data[price_col].pct_change()
data['Strategy Return'] = data['Signal'].shift(1) * data['Daily Return']
data['Cumulative Market'] = (1 + data['Daily Return']).cumprod()
data['Cumulative Strategy'] = (1 + data['Strategy Return']).cumprod()

# === Performance Metrics Function ===
def compute_metrics(returns, cumulative_returns):
    ann_factor = 252
    cagr = cumulative_returns.iloc[-1] ** (ann_factor / len(returns)) - 1
    volatility = returns.std() * np.sqrt(ann_factor)
    sharpe = ((returns.mean() * ann_factor) - risk_free_rate) / (returns.std() * np.sqrt(ann_factor))
    downside_std = returns[returns < 0].std() * np.sqrt(ann_factor)
    sortino = ((returns.mean() * ann_factor) - risk_free_rate) / downside_std if downside_std != 0 else np.nan
    roll_max = cumulative_returns.cummax()
    drawdown = cumulative_returns / roll_max - 1
    max_drawdown = drawdown.min()
    return {
        'CAGR': cagr,
        'Volatility': volatility,
        'Sharpe Ratio': sharpe,
        'Sortino Ratio': sortino,
        'Max Drawdown': max_drawdown
    }

# Compute metrics
sma_metrics = compute_metrics(data['Strategy Return'].dropna(), data['Cumulative Strategy'].dropna())
hold_metrics = compute_metrics(data['Daily Return'].dropna(), data['Cumulative Market'].dropna())

# === Print Comparison ===
print("\n--- Performance Comparison ---")
print(f"{'Metric':<20}{'SMAC Strategy':>20}{'Buy & Hold':>20}")
for key in sma_metrics:
    print(f"{key:<20}{sma_metrics[key]:>20.2%}{hold_metrics[key]:>20.2%}")

# === Plot ===
plt.figure(figsize=(14, 7))
plt.plot(data.index, data['Cumulative Market'], label='Buy & Hold', linestyle='-', color='gray')
plt.plot(data.index, data['Cumulative Strategy'], label='SMAC Strategy', linestyle='--', color='blue')
plt.title(f"{symbol} | SMAC vs Buy & Hold")
plt.xlabel("Date")
plt.ylabel("Cumulative Return")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()


/var/folders/qw/7xnsjt4n41z_t0qbcc21j_j80000gn/T/ipykernel_2955/4283044402.py:32: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  data['Signal'][long_window:] = np.where(
/var/folders/qw/7xnsjt4n41z_t0qbcc21j_j80000gn/T/ipykernel_2955/42830444


--- Performance Comparison ---
Metric                     SMAC Strategy          Buy & Hold
CAGR                               2.09%               8.78%
Volatility                        16.76%              29.41%
Sharpe Ratio                      -9.12%              26.13%
Sortino Ratio                     -8.55%              36.10%
Max Drawdown                     -22.42%             -33.43%
